In [1]:
### Importing libraries
import openai
import langchain
import pinecone
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Pinecone
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from sentence_transformers import SentenceTransformer
from langchain.prompts import PromptTemplate


c:\LLM_projects\llmenv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os

In [4]:
#### Lets read the dicument

def read_doc(directory):
    file_reader=PyPDFDirectoryLoader(directory)
    documents=file_reader.load()
    return documents

In [5]:
document=read_doc('documents/')
len(document)

22

In [6]:
#### Divide the docs into chinks

def chunk_data(document,chunk_size=800,chunk_over_lap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_over_lap)
    docs=text_splitter.split_documents(documents=document)
    return docs





In [7]:
doc=chunk_data(document)
len(doc)

58

In [8]:
#### EMbedding Techinique
embeddings = SentenceTransformerEmbeddings(model_name ='all-MiniLM-L6-v2')

c:\LLM_projects\llmenv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\LLM_projects\llmenv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [48]:
vectors=embeddings.embed_query('How are you')
len(vectors)

384

In [9]:
##vecotor sb in pineconp
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pinecone_api_key
p = pinecone.Pinecone(api_key=pinecone_api_key)
index=p.Index('pdfvector')
index_name='pdfvector'


In [10]:
vector=Pinecone.from_documents(doc,embeddings,index_name=index_name)

In [11]:
def retrive_query(query,k=2):
    matching_result=vector.similarity_search(query,k=k)
    return matching_result

In [12]:
from langchain.chains.question_answering import load_qa_chain


In [58]:

gen_ai_key=os.environ["GOOGLE_API_KEY"]
gen_ai_key
llm = ChatGoogleGenerativeAI(model="gemini-pro", client=genai,temperature=1)
prompt_template = """
    Answer the question as detailed as possible from the provided Context below, make sure to provide all the details, if the answer is not in
    provided context just say, "Answer is not available in the PDF", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """
prompt = PromptTemplate(template=prompt_template,
                            input_variables=["context", "question"])

In [35]:
prompt

PromptTemplate(input_variables=['context', 'question'], template='\n    Answer the question from context, make sure to provide all the details,\n\n\n    Context:\n {context}?\n\n    Question: \n{question}\n\n\n    Answer:\n    ')

In [59]:
chain=load_qa_chain(llm=llm,chain_type="stuff",prompt=prompt,verbose=True)

In [60]:
####Search answer from vectorDB

def retrive_answer(query):
    doc_search=retrive_query(query)
    
    print(doc_search)
    print(query)
    response=chain.run(input_documents=doc_search, question=query,return_only_outputs=True)
    return response

In [67]:
our_query="team size"

answer=retrive_answer(our_query)

print(answer)

[Document(page_content='Cricket  \n \n2 \n \n \nTeam Size  \nIn cricket, ea ch playing side has 11 players and o ne of them is appointed as \ncaptain. Apart from these 11 players, there are a few more players on each side \nwho can only field as substitute for  an injured team member . The fielding tea m \nshould have 11 players and the opposition can send only two batsmen on the \nground at the time of play.   \n \nA 15-member squad is a must for all international tourname nts conducted by the \ncricket’s governing body, International Cricket Co uncil (ICC).', metadata={'page': 5.0, 'source': 'documents\\cricket.pdf'}), Document(page_content='team. Further in the document, one can closely understand the game, its popular \nterms and rules.  \n \n \n \n1. CRICKET  – A BAT & BALL GAME', metadata={'page': 4.0, 'source': 'documents\\cricket.pdf'})]
team size


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    Answer the question a

In [70]:
print(genai.list_models())

<generator object list_models at 0x000002E082A8C640>
